In [ ]:
#Libs
import folium
import geopandas as gpd
import json
import geobr
from folium.plugins import Geocoder


In [ ]:
#Geobr 
ita = geobr.read_municipality(code_muni = 3131901)
ita.crs= 'EPSG:4326'

In [ ]:
#Reading shapefiles
risco_en = gpd.read_file('Data/risco enchente/risco_enchente.shp')
vocorocas = gpd.read_file('Data/Voçorocas/vocorocas.shp')
risco_geo = gpd.read_file('Data/Risco Geológico/risco_geo.shp')
zoneamento = gpd.read_file('Data/Zoneamento/zoneamento3.shp')
bairros = gpd.read_file('Data/Bairros/bairros.shp')
uc = gpd.read_file('Data/UC/uc.shp')
barragens = gpd.read_file('Data/Barragens/barragem.shp')
app=gpd.read_file('Data/APP/app.shp')
app['Cidade'] = 'Itabirito'
pe = gpd.read_file('Data/Parque Ecológico/PE.shp')
hidro = gpd.read_file('Data/Hidrografia/hidro.shp')
hidro.crs= 'EPSG:4326'
app_na = gpd.read_file('Data/app_nascentes/app_na.shp')
av = gpd.read_file('Data/areasverdes/av.gpkg')
na = gpd.read_file('Data/Nascentes/na.shp')

In [ ]:
#making functions
#polygons and lines
def fg_polygon(gdf,name1:str, popup, columm,fill_color:str, color:str, show1 = False):
    if show1 == True:
        fg1= folium.FeatureGroup(name=name1, show=True)
    else:
        fg1= folium.FeatureGroup(name=name1, show=False)

    gdf[columm]=gdf[columm].astype(str)
    gdf = gdf.to_crs(epsg=4326)
    

    for _, r in gdf.iterrows():

        sim_geo = gpd.GeoSeries(r['geometry'])
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': fill_color, 'color':color})
        folium.Popup(popup + r[columm]).add_to(geo_j)
        fg1.add_child(geo_j)

    return fg1.add_to(m)

#points
def fg_point(gdf,name1:str, columm ,radius1 ,weight1, color1):

    fg1= folium.FeatureGroup(name=name1, show=False)

    gdf[columm]=gdf[columm].astype(str)
    gdf = gdf.to_crs(epsg=4326)

    for _, r in gdf.iterrows():
        sim_geo = gpd.GeoSeries(r['geometry'])
        geo_j = sim_geo.to_json()
        geo_j_dict = json.loads(geo_j)
        for feature in geo_j_dict['features']:
            geometry = feature["geometry"]["coordinates"]
            a = tuple(geometry)
            if len(a) == 1:
                
                b = tuple(a[0])
                if len(b) == 2:
                    c = (b[1], b[0])
                    geo_j = folium.Circle(location=c ,radius=radius1, weight=weight1, color=color1 )
                else:
                    c = tuple(b[0])
                    d = (c[1], c[0])
                    geo_j = folium.Circle(location=d ,radius=radius1, weight=weight1, color=color1 )
            else:
                c = (a[1], a[0])
                geo_j = folium.Circle(location=c ,radius=radius1, weight=weight1, color=color1 )
        folium.Popup(r[columm]).add_to(geo_j)
        fg1.add_child(geo_j)

    return fg1.add_to(m)



In [ ]:
# Create a map using Folium
m = folium.Map(location=[-20.260629, -43.804297], zoom_start=10.5,show=False, control_scale = True )



In [ ]:
#using functions
fg_polygon(ita, 'Limite Municipal(IBGE)', 'cidade: ', 'name_muni', '#000000', '#000000', show1 = True )
fg_polygon(hidro,'Hidrografia (IGAM adapt. SEMAM)', 'Nome:  ', 'NORIOCOMP', '#0000CD', '#0000CD')
fg_point(na, 'Nascentes (SEMAM)', 'NORIOCOMP', 5, 3, 'blue')
fg_polygon(app,'Áreas de Preservação Permanentes - Hidrografia (SEMAM)', 'APP ', 'Cidade', '#000000','	#76EEC6')
fg_point(app_na, 'Áreas de Preservação Permanentes - Nascentes (SEMAM)', 'NORIOCOMP', 5,2,'	#76EEC6')
fg_polygon(uc,'Unidades de Conservação (Inst. Pristino)', 'Unidade: ', 'nome_uc', 'green', 'black')
fg_polygon(pe,'Perímetro Parque Ecológico Municipal (SEMAM)', 'Área:  ', 'Área', '#000000', '#3D9140')
fg_polygon(zoneamento,'Zoneamento Municipal (SEMURB)', 'Setor: ', 'SETOR', 'orange', 'black')
fg_polygon(bairros,'Bairros Municipais (IBGE)', 'Bairro: ', 'BAIRRO', 'yellow', 'black')
fg_point(barragens,'Barragens (FEAM)' , 'nome_bar', 10, 10, 'brown'  )
fg_polygon(vocorocas, 'Mapeamento Voçorocas (SEMAM)', 'Área (m²): ', 'AREA_M2','brown','black' )
fg_polygon(risco_geo, 'Risco Geológico (Defesa Civil)', 'Área (m²): ', 'AREA_M2','brown','black')
fg_polygon(risco_en,'Risco de Inundação (Defesa Civil)', 'Área (m²): ', 'AREA_M2', 'brown', 'black')
fg_point(av, 'Áreas Verdes (SEMAM)', "fid", 10, 5, 'green' )



In [ ]:
# Add Extra Tools
#Google Basemap 
tiles = folium.TileLayer(
        tiles ='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = False,
        control = True,
        control_scale = True).add_to(m)

# Layer Control
folium.LayerControl().add_to(m)

# Distance Measure
folium.plugins.MeasureControl(position='topleft').add_to(m)


# Search bar

Geocoder(collapsed=True, position='topleft').add_to(m)


In [ ]:
#Saving.html

m.save("Webmap_itabirito.html")